<h1> Import </h1>

In [1]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold 
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import dataset
model_trainylog = pd.read_csv('cleanedtrainwithYlog.csv')
model_test = pd.read_csv('cleanedtest.csv')

In [3]:
model_trainylog.head()

,Unnamed: 0,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,ylogSalePrice
0,0,6.684507,6.831328,20.212182,3.440268,3.055642,15.187527,15.187527,9.059126,12.170327,...,0,0,0,1,0,0,0,1,0,12.247699
1,1,4.858807,7.221214,20.712205,3.259674,3.602594,15.145138,15.145138,1.000000,13.062832,...,0,0,0,1,0,0,0,1,0,12.109016
2,2,6.684507,6.914940,21.347241,3.440268,3.055642,15.184404,15.185966,8.646538,11.200343,...,0,0,0,1,0,0,0,1,0,12.317171
3,3,6.968981,6.684507,20.691553,3.440268,3.055642,15.047529,15.135652,1.000000,9.274266,...,0,0,0,1,0,0,0,0,0,11.849405
4,4,6.684507,7.314735,22.325160,3.602594,3.055642,15.182841,15.182841,10.391827,11.971129,...,0,0,0,1,0,0,0,1,0,12.429220


<h1> Train and Test Data Split </h1>

In [4]:
X = model_trainylog.drop(['ylogSalePrice'], axis = 1)
y = model_trainylog['ylogSalePrice']

In [5]:
#Partition the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=.2)

<h1> Modeling </h1>

In [6]:
# Stratified KFold
stratify_divide = StratifiedKFold(n_splits=10, shuffle=True, random_state=99)

In [7]:
# KFold for Cross Validation
kf = KFold(n_splits = 5, shuffle = True, random_state = 28)

In [8]:
#RMSE_CV
def rmse_cv(model):
    kf = KFold(n_splits = 5, shuffle=True, random_state=42).get_n_splits(X_train)
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

<h1> Gradient Boosting Regression </h1>

In [9]:
gbm = GradientBoostingRegressor()
gbm.set_params(random_state=42)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [10]:
# Train model on training data
gbm.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [11]:
print("The train set R^2 is: %.5f" % gbm.score(X_train, y_train))
print("The test set R^2 is is: %.5f" % gbm.score(X_test, y_test))

The train set R^2 is: 0.96555
The test set R^2 is is: 0.89624


In [12]:
# Initial prediction
gbm_pred = gbm.predict(X_test)

# Calculate the absolute errors
errors = abs(np.expm1(gbm_pred) - np.expm1(y_test))

# Print out MAE, MSE, and RMSE
print('Mean Absolute Error (MAE): $', round(np.mean(errors), 2))
print('Mean Squared Error (MSE):', mean_squared_error(y_test, gbm_pred))
print('Root Mean Square Error (RMSE):', np.sqrt(mean_squared_error(y_test, gbm_pred)))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / np.expm1(y_test))
print('Mean Absolute Percent Error (MAPE):', round(np.mean(mape), 2), '%.')

Mean Absolute Error (MAE): $ 14720.51
Mean Squared Error (MSE): 0.01638708223310358
Root Mean Square Error (RMSE): 0.1280120394068604
Mean Absolute Percent Error (MAPE): 9.11 %.


<h4> Tune Hyperparameters </h4>

In [13]:
#parameter grids
gbm_param_grid={'n_estimators':[300, 400, 500, 600],
                'learning_rate':[0.05, 0.1, 1.5],
                'max_depth':[1, 2, 3],
                'min_samples_leaf':[1, 2, 3, 4]}

In [14]:
grid_search_gbm = GridSearchCV(gbm, gbm_param_grid, scoring='neg_mean_squared_error', 
                                     cv= kf, n_jobs=-1, return_train_score = True, verbose = 1)

%time  grid_search_gbm.fit(X_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  7.4min finished


CPU times: user 5.59 s, sys: 329 ms, total: 5.92 s
Wall time: 7min 26s


GridSearchCV(cv=KFold(n_splits=5, random_state=28, shuffle=True),
             error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_wei...
                                                 presort='auto',
                                                 

In [15]:

# get the best parameters
grid_search_gbm.best_params_

{'learning_rate': 0.1,
 'max_depth': 2,
 'min_samples_leaf': 3,
 'n_estimators': 600}

In [16]:
grid_gbm = grid_search_gbm.best_estimator_

In [17]:
#Prediction with tuned hyperparameters
grid_gbm_pred = grid_gbm.predict(X_test)

# Calculate the absolute errors
errors = abs(np.expm1(grid_gbm_pred) - np.expm1(y_test))

# Print out MAE, MSE, and RMSE
print('Mean Absolute Error (MAE): $', round(np.mean(errors), 2))
print('Mean Squared Error (MSE):', mean_squared_error(y_test, grid_gbm_pred))
print('Root Mean Square Error (RMSE):', np.sqrt(mean_squared_error(y_test, grid_gbm_pred)))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / np.expm1(y_test))
print('Mean Absolute Percent Error (MAPE):', round(np.mean(mape), 2), '%.')

Mean Absolute Error (MAE): $ 13960.24
Mean Squared Error (MSE): 0.015462094678347665
Root Mean Square Error (RMSE): 0.12434667136014403
Mean Absolute Percent Error (MAPE): 8.7 %.


In [18]:
score = rmse_cv(grid_gbm)
print("\nGradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Gradient Boosting score: 0.1188 (0.0111)



In [19]:
gbm_param_grid1={'n_estimators':[1000, 2000, 3000],
                'learning_rate':[0.01, 0.05, 0.1],
                'max_depth':[2, 3, 4],
                'min_samples_leaf':[5, 10, 15],
                'min_samples_split': [5, 10],
                'loss':['huber']}

In [20]:
grid_search_gbm1 = GridSearchCV(gbm, gbm_param_grid1, scoring='neg_mean_squared_error', 
                                     cv= kf, n_jobs=-1, return_train_score = True, verbose = 1)

%time  grid_search_gbm1.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 38.0min


KeyboardInterrupt: 

In [21]:
# get the best parameters
grid_search_gbm1.best_params_ 

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
grid_gbm1 = grid_search_gbm1.best_estimator_

In [ ]:
#Prediction with tuned hyperparameters
grid_gbm_pred1 = grid_gbm1.predict(X_test)

# Calculate the absolute errors
errors = abs(np.expm1(grid_gbm_pred1) - np.expm1(y_test))

# Print out MAE, MSE, and RMSE
print('Mean Absolute Error (MAE): $', round(np.mean(errors), 2))
print('Mean Squared Error (MSE):', mean_squared_error(y_test, grid_gbm_pred1))
print('Root Mean Square Error (RMSE):', np.sqrt(mean_squared_error(y_test, grid_gbm_pred1)))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / np.expm1(y_test))
print('Mean Absolute Percent Error (MAPE):', round(np.mean(mape), 2), '%.')

In [ ]:
score = rmse_cv(grid_gbm1)
print("\nGradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
#feature importance
# Get numerical feature importances
importances_gbm = list(grid_gbm.feature_importances_)

# List of tuples with variable and importance
feature_importances_gbm = [(feature, round(importance, 5)) for feature, importance in zip(X_train.columns, importances_gbm)]

# Sort the feature importances by most important first
gbm_feature_importances = sorted(feature_importances_gbm, key = lambda x: x[1], reverse = True )

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in gbm_feature_importances]

gbm_feature_importances_top20 = gbm_feature_importances[:20]
featureNames, featureScores = zip(*list(gbm_feature_importances_top20))

In [ ]:
plt.barh(range(len(featureScores)), featureScores, tick_label=featureNames)
plt.title('feature importance')
plt.gca().invert_yaxis()
plt.ylabel('Features')
plt.xlabel('Importance Score')
plt.title('Feature Importances')

In [ ]:

y_train_gradientboost = grid_gbm.predict(X_train)
y_test_gradientboost = grid_gbm.predict(X_test)

# Plot predictions
plt.figure(figsize=(12,8))
plt.scatter(np.expm1(y_train_gradientboost), np.expm1(y_train), c='black', marker="o", s=15, label = "Training data")
plt.scatter(np.expm1(y_test_gradientboost), np.expm1(y_test), c='orange', marker='o', s=15, label = "Validation data")
plt.title("Gradient Boosting", fontsize = 20)
plt.xlabel("Predicted Prices", fontsize = 16)
plt.ylabel("Actual Prices", fontsize = 16)
plt.xlim(0, 800000)
plt.ylim(0, 800000)
plt.legend(loc = "upper left")
plt.plot([0, 800000], [0, 800000], c = "grey")
plt.show()

In [ ]:
gbm_params_tuned_model = grid_search_gbm.best_estimator_
gbm_feature_importance = 100.0 * (gbm_params_tuned_model.feature_importances_ / gbm_params_tuned_model.feature_importances_.max())
gbm_important_features = X_train.columns[gbm_feature_importance >= 1]
gbm_unimportant_features = X_train.columns[gbm_feature_importance < 1]

In [ ]:
X_train_gbmreduced = X_train.drop(gbm_unimportant_features, axis=1)
X_test_gbmreduced = X_test.drop(gbm_unimportant_features, axis=1)

In [ ]:
gbm_feats =GradientBoostingRegressor()

gbm_feats.set_params(random_state=42, verbose =1, learning_rate = 0.1, max_depth = 2, min_samples_leaf = 1, n_estimators = 500)

In [ ]:
# GridSearchCV for multiple hyperparameters:
gbm_param_grid_feats={'n_estimators':[100, 500],
                      'learning_rate':[0.05, 0.1]
                      } 
grid_search_gbm2 = GridSearchCV(gbm_feats, gbm_param_grid_feats, scoring= 'neg_mean_squared_error',
                           cv= kf, n_jobs = 7, return_train_score=True, verbose = 1)
grid_search_gbm2.fit(X_train_gbmreduced, y_train)

In [ ]:
# Use the Gradient Boost's predict method on the test data
predictions_tuned_gbm2 = grid_search_gbm2.best_estimator_.predict(X_test_gbmreduced)

# Calculate the absolute errors
errors = abs(np.expm1(predictions_tuned_gbm2) - np.expm1(y_test))

# Print out the mean absolute error (MAE)
print('Mean Absolute Error (MAE): $', round(np.mean(errors), 2))
print('MSE:', mean_squared_error(y_test, predictions_tuned_gbm2))
print('RMSE:', np.sqrt(mean_squared_error(y_test, predictions_tuned_gbm2)))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / np.expm1(y_test))

# Calculate and display MAPE
#accuracy = 100 - np.mean(mape)
print('MAPE:', round(np.mean(mape), 2), '%.')

In [ ]:
score = rmse_cv(grid_gbm2)
print("\nGradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
#0.2 * grid_gbm1.predict(X_test)